# LecoSpec Image Segmentation

In [2]:
library(raster)

Warning message:
"package 'raster' was built under R version 4.1.1"
Loading required package: sp

Warning message:
"package 'sp' was built under R version 4.1.1"


In [3]:
library(itcSegment)
test <- "some string"
test_2 <- "another String"

Loading required package: maptools

Warning message:
"package 'maptools' was built under R version 4.1.1"
Checking rgeos availability: TRUE
Please note that 'maptools' will be retired by the end of 2023,
plan transition at your earliest convenience;
some functionality will be moved to 'sp'.

Loading required package: rgeos

Warning message:
"package 'rgeos' was built under R version 4.1.1"
rgeos version: 0.5-7, (SVN revision 676)
 GEOS runtime version: 3.9.1-CAPI-1.14.2 
 Please note that rgeos will be retired by the end of 2023,
plan transition to sf functions using GEOS at your earliest convenience.
 GEOS using OverlayNG
 Linking to sp version: 1.4-5 
 Polygon checking: TRUE 




In [5]:
source("./Scripts/Segmentation/crop_and_classify.R")

Loading required package: rgdal

Warning message:
"package 'rgdal' was built under R version 4.1.1"
Please note that rgdal will be retired by the end of 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.

rgdal: version: 1.5-25, (SVN revision 1143)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/kenne/Documents/R/win-library/4.1/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/kenne/Documents/R/win-library/4.1/rgdal/proj
PROJ CDN enabled: FALSE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading sp or rgdal.
Overwritten PROJ_LIB was C:/Users/kenne/Documents/R/win-library/4.1/rgdal/proj

Loading required package: s

In [6]:
epsg_code <- 26983
target_wkt <- sf::st_crs(epsg_code)[[2]]
target_crs <- sp::CRS(target_wkt)

In [7]:
print(getwd())

[1] "c:/Users/kenne/Documents/GitHub/Tree_Spectral_Library"


In [27]:
base_tile_path <- paste0(
    getwd(),
    "/Data/raw_15366_rd_rf_or/Tiles/A_001_raw_15366_rd_rf_or_Tile_"
    )
tile_paths <- c(
    paste0(base_tile_path, 16, ".envi"),
    paste0(base_tile_path, 17, ".envi"),
    paste0(base_tile_path, 18, ".envi"),
    paste0(base_tile_path, 19, ".envi"),
    paste0(base_tile_path, 20, ".envi")
)
print(tile_paths)

# define function to merge the tiles and extract the correct band
# should live in general lecospec
merge_tiles <- function(input_files, output_path = NA) {

    master_raster <- raster::brick(input_files[[1]])
        #print("Extent of first tile:")
        #print(raster::extent(master_raster))

    for (input_file in tail(input_files, -1)) {
        new_raster <- raster::brick(input_file)
        #print("Extent of tile:")
        #print(raster::extent(new_raster))
        master_raster <- raster::merge(
            master_raster,
            new_raster,
            tolerance = 0.5)
        
    }
    if(!is.na(output_path)) {
        raster::writeRaster(master_raster, output_path)
    }

    return(master_raster)
}

# load the data from the relevant tiles
test_raster <- merge_tiles(
    tile_paths,
    output_path = "./Outputs/merged_tiles.tif")




[1] "c:/Users/kenne/Documents/GitHub/Tree_Spectral_Library/Data/raw_15366_rd_rf_or/Tiles/A_001_raw_15366_rd_rf_or_Tile_16.envi"
[2] "c:/Users/kenne/Documents/GitHub/Tree_Spectral_Library/Data/raw_15366_rd_rf_or/Tiles/A_001_raw_15366_rd_rf_or_Tile_17.envi"
[3] "c:/Users/kenne/Documents/GitHub/Tree_Spectral_Library/Data/raw_15366_rd_rf_or/Tiles/A_001_raw_15366_rd_rf_or_Tile_18.envi"
[4] "c:/Users/kenne/Documents/GitHub/Tree_Spectral_Library/Data/raw_15366_rd_rf_or/Tiles/A_001_raw_15366_rd_rf_or_Tile_19.envi"
[5] "c:/Users/kenne/Documents/GitHub/Tree_Spectral_Library/Data/raw_15366_rd_rf_or/Tiles/A_001_raw_15366_rd_rf_or_Tile_20.envi"


ERROR: Error in rgdal::putRasterData(x@file@transient, vv, band = i, offset = off): Failure during raster IO



In [19]:
full_datacube_path <- "E:/lecospec/raw_2000_rd_rf_or"
full_cube <- raster::brick(full_datacube_path)

CRS arguments: +proj=longlat +datum=WGS84 +no_defs 


In [22]:
print("CRS and extent of Test Raster")
print(crs(test_raster))
print(extent(test_raster))

print("CRS and extent of Full Datacube")
print(crs(full_cube))
print(extent(full_cube))


[1] "CRS and extent of Test Raster"
CRS arguments: +proj=longlat +datum=WGS84 +no_defs 
class      : Extent 
xmin       : -71.14266 
xmax       : -71.14231 
ymin       : 42.81763 
ymax       : 42.81883 
[1] "CRS and extent of Full Datacube"
CRS arguments: +proj=longlat +datum=WGS84 +no_defs 
class      : Extent 
xmin       : -68.62562 
xmax       : -68.62446 
ymin       : 44.84502 
ymax       : 44.84654 


In [ ]:
cropped_raster <- raster::crop(
    test_raster,
    full_cube,
    filename = "cropped_input.tif")

In [ ]:
segmentation_path <- "E:/lecospec/Data/seg/"
manual_segmentation <- rgdal::readOGR(
    segmentation_path,
    verbose = TRUE,
    layer = "crown_delims")
